In [ ]:
##This notebook is built around using tensorflow as the backend for keras
#!KERAS_BACKEND=tensorflow python -c "from keras import backend"

In [2]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
import mxnet as mx

from mxnet import gluon, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms

import gluoncv
from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, TrainingHistory
from gluoncv.data import transforms as gcv_transforms

In [4]:
# GENERAL PARAMETERS
IMG_WIDTH, IMG_HEIGHT = 299, 299

train_data_dir = 'Open_I_abd_vs_CXRs/TRAIN' #location of training data
validation_data_dir = 'Open_I_abd_vs_CXRs/VAL' #location of validation data
# number of samples used for determining the samples_per_epoch
nb_train_samples = 65
nb_validation_samples = 10

EPOCHS = 5
BATCH_SIZE = 5  

In [5]:

# TENSORFLOW 
train_datagen = ImageDataGenerator(
        rescale=1./255,            # normalize pixel values to [0,1]
        shear_range=0.2,      
        zoom_range=0.2,    
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)  


val_datagen = ImageDataGenerator(
         rescale=1./255)       # normalize pixel values to [0,1]

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 65 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [6]:
#!python3 -m pip uninstall d2l
#help(gluon.data.vision.transforms)

In [7]:
# MXNET

train_imgs = gluon.data.vision.ImageFolderDataset(train_data_dir)
test_imgs = gluon.data.vision.ImageFolderDataset(validation_data_dir)

train_augs = gluon.data.vision.transforms.Compose([
    gluon.data.vision.transforms.RandomResizedCrop(224),
    gluon.data.vision.transforms.RandomFlipLeftRight(),
    gluon.data.vision.transforms.ToTensor()
])

test_augs = gluon.data.vision.transforms.Compose([
    gluon.data.vision.transforms.Resize(256),
    gluon.data.vision.transforms.CenterCrop(224),
    gluon.data.vision.transforms.ToTensor()
])
'''
transform_train = transforms.Compose([
    gcv_transforms.RandomCrop(32, pad=4),
    transforms.RandomFlipLeftRight(),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

# Define the data loaders for the training and test dataset.
train_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10(train=True).transform_first(
        transform_train),  # set path to the downloaded data
    batch_size=batch_size, shuffle=True, last_batch='discard', num_workers=num_workers)

val_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10(train=False).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)
'''
train_data_mx = gluon.data.DataLoader(
    train_imgs.transform_first(train_augs), BATCH_SIZE, shuffle=True)
#test_iter = gluon.data.DataLoader(
#    test_imgs.transform_first(test_augs), batch_size)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Ignoring Open_I_abd_vs_CXRs/TRAIN/.DS_Store, which is not a directory.
  This is separate from the ipykernel package so we can avoid doing imports until


## GPU - Reproducibility

In [8]:
# MXNET
gpus = mx.test_utils.list_gpus()
ctx =  [mx.gpu()] if gpus else [mx.cpu(0), mx.cpu(1)]
print(ctx)

[cpu(0), cpu(1)]


## Get pretrained model

In [9]:
# TENSORFLOW
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [10]:
# MXNET
net = gluoncv.model_zoo.I3D_InceptionV3(pretrained=True, ctx = ctx, nclass=2)

## Attach last layer

In [11]:
# MXNET
deep_medical_net = gluoncv.model_zoo.I3D_InceptionV3(prefix='deep_medical_', nclass=2)
deep_medical_net.collect_params().initialize(ctx=ctx, force_reinit=True)
deep_medical_net.features = net.features
deep_medical_net.output.initialize(mx.init.Xavier(), force_reinit=True)
deep_medical_net.summary

deep_medical_conv0_weight is done with shape:  (32, 3, 3, 3, 3)
deep_medical_batchnorm0_gamma is done with shape:  (32,)
deep_medical_batchnorm0_beta is done with shape:  (32,)
deep_medical_batchnorm0_running_mean is done with shape:  (32,)
deep_medical_batchnorm0_running_var is done with shape:  (32,)
deep_medical_conv1_weight is done with shape:  (32, 32, 3, 3, 3)
deep_medical_batchnorm1_gamma is done with shape:  (32,)
deep_medical_batchnorm1_beta is done with shape:  (32,)
deep_medical_batchnorm1_running_mean is done with shape:  (32,)
deep_medical_batchnorm1_running_var is done with shape:  (32,)
deep_medical_conv2_weight is done with shape:  (64, 32, 3, 3, 3)
deep_medical_batchnorm2_gamma is done with shape:  (64,)
deep_medical_batchnorm2_beta is done with shape:  (64,)
deep_medical_batchnorm2_running_mean is done with shape:  (64,)
deep_medical_batchnorm2_running_var is done with shape:  (64,)
deep_medical_conv3_weight is done with shape:  (80, 64, 1, 1, 1)
deep_medical_batchnor

<bound method Block.summary of I3D_InceptionV3(
  (features): HybridSequential(
    (0): HybridSequential(
      (0): Conv3D(3 -> 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 0, 0), bias=False)
      (1): BatchNorm(axis=1, eps=0.001, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=32)
      (2): Activation(relu)
    )
    (1): HybridSequential(
      (0): Conv3D(32 -> 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
      (1): BatchNorm(axis=1, eps=0.001, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=32)
      (2): Activation(relu)
    )
    (2): HybridSequential(
      (0): Conv3D(32 -> 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (1): BatchNorm(axis=1, eps=0.001, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
      (2): Activation(relu)
    )
    (3): MaxPool3D(size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 0, 0), ceil_mode=False, global_pool=Fals

In [12]:
# TENSORFLOW
model_top = Sequential()
model_top.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:], data_format=None)),  
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid')) 

model = Model(inputs=base_model.input, outputs=model_top(base_model.output))

model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0), loss='binary_crossentropy', metrics=['accuracy'])

## Train

In [ ]:
# TENSORFLOW
history = model.fit(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)


In [14]:
# MXNET
from mxnet import autograd

# Saved in the d2l package for later use
def training_procedure(handwritten_net, train_iter):
    global EPOCHS
    global ctx
    handwritten_net.initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)
    #handwritten_net(init = mx.init.Xavier(), ctx=ctx)
    optim = mx.optimizer.Adam(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, lazy_update=True)
    trainer = gluon.Trainer(handwritten_net.collect_params(), optim)
    # Use Accuracy as the evaluation metric.
    metric = mx.metric.Accuracy()
    softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
    
    
    for i in range(EPOCHS):
        # Loop over the train data iterator.
        for i, (features, labels) in enumerate(train_iter):
        #for batch in train_data:
            # Splits train data into multiple slices along batch_axis
            # and copy each slice into a context.
            ###data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            # Splits train labels into multiple slices along batch_axis
            # and copy each slice into a context.
            ###label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            # Inside training scope
            with autograd.record():
                for x, y in zip(features, labels):
                    print(x.shape)
                    #help(handwritten_net)
                    z = handwritten_net(x)
                    # Computes softmax cross entropy loss.
                    loss = softmax_cross_entropy_loss(z, y)
                    # Backpropogate the error for one iteration.
                    loss.backward()
                    outputs.append(z)
            # Updates internal evaluation
            metric.update(labels, outputs)
            # Make one step of parameter update. Trainer needs to know the
            # batch size of data to normalize the gradient by 1/batch_size.
            trainer.step(features.shape[0])
        # Gets the evaluation result.
        name, acc = metric.get()
        # Reset evaluation result to initial state.
        metric.reset()
        print('training acc at epoch %d: %s=%f'%(i, name, acc))
    return handwritten_net

trained_model_mx = training_procedure(deep_medical_net, train_data_mx)

(3, 224, 224)


MXNetError: [18:10:52] src/operator/nn/convolution.cc:211: Check failed: dshp.ndim() == 5U (3 vs. 5) : Input data should be 5D in batch-num_filter-depth-y-x
Stack trace:
  [bt] (0) 1   libmxnet.so                         0x00000001503ed929 mxnet::op::NDArrayOpProp::~NDArrayOpProp() + 4473
  [bt] (1) 2   libmxnet.so                         0x000000015071afbf void mxnet::op::ConcatCSRImpl<mshadow::cpu>(nnvm::NodeAttrs const&, mxnet::OpContext const&, std::__1::vector<mxnet::NDArray, std::__1::allocator<mxnet::NDArray> > const&, std::__1::vector<mxnet::OpReqType, std::__1::allocator<mxnet::OpReqType> > const&, std::__1::vector<mxnet::NDArray, std::__1::allocator<mxnet::NDArray> > const&) + 141199
  [bt] (2) 3   libmxnet.so                         0x000000015196af23 mxnet::imperative::SetShapeType(mxnet::Context const&, nnvm::NodeAttrs const&, std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArray*> > const&, std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArra

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())

plt.figure()
plt.plot(history.history['accuracy'], 'orange', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'blue', label='Validation accuracy')
plt.plot(history.history['loss'], 'red', label='Training loss')
plt.plot(history.history['val_loss'], 'green', label='Validation loss')
plt.legend()
plt.show()


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path='Open_I_abd_vs_CXRs/TEST/chest2.png' #change to location of chest x-ray
img_path2='Open_I_abd_vs_CXRs/TEST/abd2.png'  #change to location of abd x-ray
img = image.load_img(img_path, target_size=(img_width, img_height))
img2 = image.load_img(img_path2, target_size=(img_width, img_height))
plt.imshow(img)
plt.show()

img = image.img_to_array(img)
x = np.expand_dims(img, axis=0) * 1./255
score = model.predict(x)
print('Predicted:', score, 'Chest X-ray' if score < 0.5 else 'Abd X-ray')

plt.imshow(img2)
plt.show()

img = image.img_to_array(img2)
x = np.expand_dims(img2, axis=0) * 1./255
score2 = model.predict(x)
print('Predicted:', score2, 'Chest X-ray' if score2 < 0.5 else 'Abd X-ray')
